In [10]:
import requests
from bs4 import BeautifulSoup
import os, re
from openai import OpenAI
import time  
from datetime import datetime  


: 

# 'https://menet.com.cn

In [13]:
host ='https://www.menet.com.cn'


In [15]:
def get_list():
    host ='https://www.menet.com.cn/news'
    response = requests.post(host)
    soup = BeautifulSoup(response.text, 'html.parser')
    news_list = soup.find_all('h3')
    return soup, news_list


In [17]:
soup, news_list = get_list()


In [19]:
news_list = [_ for _ in news_list if "登录" not in _.text]


In [21]:
def get_info(item, host ='https://www.menet.com.cn'):
    link = host + item.a['href']
    title = item.text
    date = item.a['href'].split('/')[3][:8]
    return {
    "title": title,
    'url': link,
    'date':date
}


In [ ]:
result1 = get_info()


In [ ]:
for num, item in enumerate(soup.find_all(lambda tag: tag.has_attr('class') and len(tag)==1 and tag['class'] == ["item_time"])):
    result1[num]['date'] = item.text


# https://www.pharnexcloud.com/zixun

In [18]:
r = requests.get('https://www.pharnexcloud.com/zixun')


In [19]:
soup = BeautifulSoup(r.text, 'html.parser')


In [20]:
dates = soup.find_all(lambda tag:tag.has_attr('class') and tag['class']==['date'])


In [22]:
result = soup.find_all('a', class_ = 'img-content')


In [28]:
def query_tilte_url(item):
    title = item.find_next()['alt']
    url = 'https://www.pharnexcloud.com' + item['href']
    return {
        'title': title,
        'url': url
    }


In [30]:
res = list(map(query_tilte_url, result))


In [32]:
for no, item in enumerate(res):
    
    item['date'] = dates[no]


In [34]:
result2 = res


---

# https://bydrug.pharmcube.com/

In [8]:
r = requests.get('https://bydrug.pharmcube.com/')


In [9]:
soup = BeautifulSoup(r.text, 'html.parser')


In [10]:
datas = soup.find_all('div', class_= 'mf-flex mf-items-center')


In [14]:
def get_tilte_and_url(item):
    title = item.text.strip()
    url = item.a['href']
    return {
        'title': title,
        'url': url
    }


In [ ]:
res = list(map(get_tilte_and_url, datas))


In [50]:
for no, _ in enumerate(res):
    _['date'] = dates[no]


In [52]:
result3 = res


In [54]:
import pandas as pd


In [56]:
df = pd.DataFrame(result1+result2+result3)


---

In [60]:
# 用r.jina转换成ai友好格式，并用通义千问进行总结


In [66]:
def summarize(url):
    context = requests.get('https://r.jina.ai/'+url).text
    url_pattern = re.compile(r'\bhttps?:\/\/[^\s/$.?#].[^\s]*\b')  
    cleaned_text = url_pattern.sub('', context)
    context = cleaned_text[:6000] if len(cleaned_text)>6000  else cleaned_text
    context = "请尽量简洁总结以下内容，并发表你的观点,文字不得超过150字\n"+context.replace("\n","")[:6000]
    client = OpenAI(
    api_key='sk-69c9bf69fa2b4d2985834d6dc17c0790', # 如何获取API Key：https://help.aliyun.com/zh/model-studio/developer-reference/get-api-key
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )
    completion = client.chat.completions.create(
        model="qwen-max", # 更多模型请参见模型列表文档。
        messages=[{"role": "user", "content": context}]
    )
    print(completion.choices[0].message.content)
    with open('output.md', 'w') as f:  
        # 使用 print 函数，将输出重定向到文件  
        print(completion.choices[0].message.content, file=f)  
        
    return completion.choices[0].message.content


In [ ]:
df


In [70]:
result = result1 + result2 + result3


In [72]:
def get_and_send(item):
    url = item['url']
    title= item['title']
    summary = summarize(url)
    return summary


In [74]:
li = result = result1 + result2 + result3


In [ ]:
li[3]


In [150]:
title, url, date = li[3].values()


In [ ]:
title


In [ ]:
url


In [ ]:
date


In [411]:
l = li[:10]


In [ ]:
summary = get_and_send(li[3])


In [ ]:
l


In [ ]:
for k, v in l[0].items():
    print(k, v)


In [616]:
no = [str(_)+'.' for _ in range(1, 11)] 


In [680]:
li = [f"[{_['title']}]({_['url']})" for _ in l ]


In [682]:
li = [b[:1] + a+ b[1:] for a, b in zip(no, li)]


In [ ]:
li


In [686]:
li = [_ + '\n\n' for _ in li ]


In [ ]:
li


In [670]:
url2 = 'https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key=660ed8dd-be77-4f1b-8de8-774d53f1f39c'


In [637]:
send_markdown_to_wechat(f"# [{title}]({url})", )


In [638]:
send_markdown_to_wechat(f"### [{title}]({url})", url )


In [180]:
link = f"### [{title}]({url})"


In [ ]:
title


In [172]:
link+= '/n'


In [ ]:
link


In [242]:
send_card_to_wechat()


In [401]:
def send_markdown_to_wechat(msg='test', webhook_url='https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key=a0ad9b92-b9a5-43d3-a129-98c954ecb159'):
    
    headers = {  
        'Content-Type': 'application/json',  
    }

    data = dic
   
    response = requests.post(webhook_url, json=data, headers=headers)


In [672]:
send_markdown_to_wechat(msg = "".join(li))


In [690]:
send_markdown_to_wechat(msg = "".join(li), webhook_url='https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key=660ed8dd-be77-4f1b-8de8-774d53f1f39c')


In [ ]:
s
